## Prepare Environment

In [ ]:
import gc
import os
from pathlib import Path
import random
import sys

from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import scipy as sp

import matplotlib.pyplot as plt

# --- plotly ---
from pydicom import dcmread
import pickle
import cv2

### Install Detectron

In [ ]:
# if you are running on cpu
!pip install detectron2 -f "https://dl.fbaipublicfiles.com/detectron2/wheels/cpu/torch1.7/index.html"

In [ ]:
# if you are running on gpu
!pip install detectron2 -f "https://dl.fbaipublicfiles.com/detectron2/wheels/cu102/torch1.7/index.html"

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog, transforms, DatasetMapper, build_detection_train_loader
from detectron2.structures import BoxMode
from detectron2.evaluation import COCOEvaluator
from detectron2.data.samplers import RepeatFactorTrainingSampler

%matplotlib inline


## Data Import

In [ ]:
# Path to dataframe and image folders
PATH_ORIGIN = "../input/vinbigdata-chest-xray-abnormalities-detection"
PATH_RESIZED = "../input/vinbigdata-chest-xray-resized-png-256x256"
len(os.listdir(os.path.join(PATH_ORIGIN, "train")))

In [ ]:
train_dataframe = pd.read_csv(os.path.join(PATH_ORIGIN, 'train.csv'))

## Data Exploration

### Data Information given by Kaggle

Images are in the DICOM format https://de.wikipedia.org/wiki/Digital_Imaging_and_Communications_in_Medicine
A .dicom file contains not just the pixel values but all sorts of usefull information

Classes 
* 0 - Aortic enlargement
* 1 - Atelectasis
* 2 - Calcification
* 3 - Cardiomegaly
* 4 - Consolidation
* 5 - ILD
* 6 - Infiltration
* 7 - Lung Opacity
* 8 - Nodule/Mass
* 9 - Other lesion
* 10 - Pleural effusion
* 11 - Pleural thickening
* 12 - Pneumothorax
* 13 - Pulmonary fibrosis
* 14 - No finding

In [ ]:
print(train_dataframe.shape)
train_dataframe.head()

### Sample .dicom File Display

In [ ]:
ds = dcmread(os.path.join(PATH_, 'train', '000434271f63a053c4128a0ba6352c7f.png'))
# Metadata readout
print(ds)
# Pixel Array display
plt.imshow(ds.pixel_array, cmap=plt.cm.gray)

## Data Preparation

In [ ]:
def get_vinbigdata_dicts(
    imgdir: Path, train: pd.DataFrame, use_cache: bool = True, debug: bool = True,
):
    debug_str = f"_debug{int(debug)}"
    cache_path = Path(".") / f"dataset_dicts_cache{debug_str}.pkl"
    if not use_cache or not cache_path.exists():
        print("Creating data...")
        train_meta = pd.read_csv(imgdir / "train_meta.csv")
        if debug:
            train_meta = train_meta.iloc[:500]  # For debug....

        # Load 1 image to get image size.
        image_id = train_meta.loc[0, "image_id"]
        image_path = str(imgdir / "train" / f"{image_id}.png")
        image = cv2.imread(image_path)
        resized_height, resized_width, ch = image.shape
        print(f"image shape: {image.shape}")

        dataset_dicts = []
        for index, train_meta_row in tqdm(train_meta.iterrows(), total=len(train_meta)):
            record = {}

            image_id, height, width = train_meta_row.values
            filename = str(imgdir / "train" / f"{image_id}.png")
            record["file_name"] = filename
            record["image_id"] = index
            record["height"] = resized_height
            record["width"] = resized_width
            objs = []
            for index2, row in train.query("image_id == @image_id").iterrows():
                # print(row)
                # print(row["class_name"])
                # class_name = row["class_name"]
                class_id = row["class_id"]
                if class_id == 14:
                    # It is "No finding"
                    # This annotator does not find anything, skip.
                    pass
                else:
                    # bbox_original = [int(row["x_min"]), int(row["y_min"]), int(row["x_max"]), int(row["y_max"])]
                    h_ratio = resized_height / height
                    w_ratio = resized_width / width
                    bbox_resized = [
                        int(row["x_min"]) * w_ratio,
                        int(row["y_min"]) * h_ratio,
                        int(row["x_max"]) * w_ratio,
                        int(row["y_max"]) * h_ratio,
                    ]
                    obj = {
                        "bbox": bbox_resized,
                        "bbox_mode": BoxMode.XYXY_ABS,
                        "category_id": class_id,
                    }
                    objs.append(obj)
            record["annotations"] = objs
            dataset_dicts.append(record)
        with open(cache_path, mode="wb") as f:
            pickle.dump(dataset_dicts, f)

    print(f"Load from cache {cache_path}")
    with open(cache_path, mode="rb") as f:
        dataset_dicts = pickle.load(f)
    return dataset_dicts


def get_vinbigdata_dicts_test(
    imgdir: Path, test_meta: pd.DataFrame, use_cache: bool = True, debug: bool = True,
):
    debug_str = f"_debug{int(debug)}"
    cache_path = Path(".") / f"dataset_dicts_cache_test{debug_str}.pkl"
    if not use_cache or not cache_path.exists():
        print("Creating data...")
        # test_meta = pd.read_csv(imgdir / "test_meta.csv")
        if debug:
            test_meta = test_meta.iloc[:500]  # For debug....

        # Load 1 image to get image size.
        image_id = test_meta.loc[0, "image_id"]
        image_path = str(imgdir / "test" / f"{image_id}.png")
        image = cv2.imread(image_path)
        resized_height, resized_width, ch = image.shape
        print(f"image shape: {image.shape}")

        dataset_dicts = []
        for index, test_meta_row in tqdm(test_meta.iterrows(), total=len(test_meta)):
            record = {}

            image_id, height, width = test_meta_row.values
            filename = str(imgdir / "test" / f"{image_id}.png")
            record["file_name"] = filename
            record["image_id"] = index
            record["height"] = resized_height
            record["width"] = resized_width
            # objs = []
            # record["annotations"] = objs
            dataset_dicts.append(record)
        with open(cache_path, mode="wb") as f:
            pickle.dump(dataset_dicts, f)

    print(f"Load from cache {cache_path}")
    with open(cache_path, mode="rb") as f:
        dataset_dicts = pickle.load(f)
    return dataset_dicts

In [ ]:
dataset_dicts = get_vinbigdata_dicts(Path(PATH_RESIZED), train_dataframe)

In [ ]:
dataset_dicts

In [ ]:
thing_classes = [
    "Aortic enlargement",
    "Atelectasis",
    "Calcification",
    "Cardiomegaly",
    "Consolidation",
    "ILD",
    "Infiltration",
    "Lung Opacity",
    "Nodule/Mass",
    "Other lesion",
    "Pleural effusion",
    "Pleural thickening",
    "Pneumothorax",
    "Pulmonary fibrosis"
]
category_name_to_id = {class_name: index for index, class_name in enumerate(thing_classes)}

In [ ]:
DatasetCatalog.register(
    "vinbigdata_train3", lambda: get_vinbigdata_dicts(Path(PATH_RESIZED), train_dataframe)
)
MetadataCatalog.get("vinbigdata_train3").set(thing_classes=thing_classes)
metadata = MetadataCatalog.get("vinbigdata_train3")
vinbig_metadata = MetadataCatalog.get("vinbigdata_train3")


In [ ]:
vinbig_metadata
for d in random.sample(dataset_dicts, 21):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=vinbig_metadata, scale=1)
    out = visualizer.draw_dataset_dict(d)
    plt.imshow(out.get_image()[:, :, ::-1]) 



In [ ]:
model_name = "COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(model_name))
cfg.DATASETS.TRAIN = ("vinbigdata_train3",)
cfg.DATALOADER.NUM_WORKERS = 0
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(model_name)  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 14

#####
# Testing here
#####
cfg.SOLVER.MAX_ITER = 5000
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 224
cfg.SOLVER.BASE_LR = 0.0005
cfg.MODEL.RPN.NMS_THRESH = 0.5


os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

# Predictions

### Setting Predictor

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained

predictor = DefaultPredictor(cfg)

## Prediction on one image

In [ ]:
from detectron2.utils.visualizer import ColorMode
dataset_dicts = val_set
d = dataset_dicts[9]
im = cv2.imread(d["file_name"])
outputs = predictor(im)  
v = Visualizer(im[:, :, ::-1],
                metadata=facemask_metadata, 
                scale=3
)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
plt_imshow(out.get_image()[:, :, ::-1])